# II. Data and Preparation

This section covers the process of restructuring the original dataset and creating additional variables. 

### Set up environment


In [3]:
# Import packages.
import numpy as np
import pandas as pd


In [2]:
# Load dataset from local file.
precip_data = pd.read_excel(r'C:\Users\15414\Documents\GitHub\MA Precip\MAPrecipData\Precipitation Database 10.19.xls')
# precip_data.head(10)


### Divide data into separate tables

Here, I divide the raw basin, station, and precipitation data into separate tables. This ensures the precipitation data tables are cleaner and easier to understand from a human perspecitive; it also allows for the concatentation of quantifiable summary variables to each categorical geographic element.


In [3]:
# Create new table from all unique rows in the basin data columns.
basin_data = precip_data[['Region', 'Basin Name']]
unique_basins = basin_data.drop_duplicates()
unique_basins = unique_basins.sort_values(by=['Region', 'Basin Name'])
# unique_basins

In [4]:
# Create new table from all unique rows in the station data columns.
station_data = precip_data[['Region', 'Basin Name', 'CITY', 'STATION']]
unique_stations = station_data.drop_duplicates()
unique_stations = unique_stations.sort_values(by=['Region', 'Basin Name'])
unique_stations = unique_stations.sort_values(by=['STATION']) # Alphabetical sorting is enforced.
# unique_stations

Within the unique basin table, there are several rows where either the basin or region has a NaN value. I remove these stations since I do not know why categorical geographic information was not applied; stations may be on the boundary of a region or basin, but is not specified.

In [5]:
# Print stations with NaN data.
print(unique_stations.loc[(unique_stations['Region'].isna() | unique_stations['Basin Name'].isna())])


          Region Basin Name              CITY STATION
1111         NaN    MILLERS   South Royalston  BIRCOE
1233         NaN  WESTFIELD         Blandford  BORNWS
2784         NaN  QUINEBAUG          Fiskdale  EBRCOE
3179         NaN        NaN       Fitzwilliam  FIT401
7917   Northeast        NaN            Newton  NEW712
8287   Northeast        NaN     North Andover  NOR550
8295         NaN        NaN  North  Attleboro  NOR800
8606   Northeast        NaN           Peabody  PEA611
11654    Western        NaN         West Otis  WES101


In [6]:
# Remove precipitation records where baisn or region is not present.
precip_data = precip_data.loc[(precip_data['Region'].notna() & 
                               precip_data['Basin Name'].notna())]
      

In [7]:
# Remove station records where baisn or region is not present.
unique_stations = unique_stations.loc[(unique_stations['Region'].notna() & 
                                       unique_stations['Basin Name'].notna())]


Finally, I will remove basin and region data from the precipitation table; this information can be accessed by indexing into the appropirate table using the station name as a key.

In [8]:
# Remove region and basin data from precipitation dataset.
precip_data = precip_data.iloc[:,:-4]
# precip_data.head(10)

### Restructure precipitation table

Sometimes I will need to access the precipitation data by the location of the cell (e.g., the cell at row one, column one) and sometimes by the higher level data associated with that value (e.g., January 1997 at station AMN307). While 2D pandas dataframes can be indexed into using both methods, I find it easier to perform the second method on a table that has been indexed at multiple levels. 

For this reason, I choose to create two equivalent, but differenlty structured precipitation tables. The first will be completely flattened with numerical index, a single column of precipitation data, and multiple columns of temporal and spatial data. The second will also feature a a single column of precipitation data, but the temporal and spatial data will set as multiple, nested indices. Currently, the data is partially flattened with each month being a separate column; I will unflatten it completely before multi-indexing.

<font size="1.2"> Technical note: I will be creating a dataframe with a multi-level index (class: pandas.core.frame.DataFrame), not a multi-index object (class: pandas.core.indexes.multi.MultiIndex). I chose this because converting to a MultiIndex object changed all NaN values to -1 and  dataframes display much more nicely and easily than MultiIndex objects. To display the precip_multi table as a MultiIndex object, use the precip_multi.index command. </font>


In [9]:
# Flatten precip data by month
precips_flat = precip_data.iloc[:,2:].unstack() # Unstack/flatten precip data only.
precips_flat = precips_flat.values.tolist()


In [10]:
# Create month list w/ numeric values. 
# This will be a new column added to the unstacked precip data.
# 1 = Jan, 2 = Feb, etc.
months_flat = len(precip_data)*[1,2,3,4,5,6,7,8,9,10,11,12]


In [11]:
# Add the station and year twelve times for every row to the appropriate list.
# These will be new columns added to the unstacked precip data.

stations_flat = []
years_flat = []

table_length = len(precip_data)

for row in range(0, table_length):
    station_name = [precip_data.iloc[row,0]] # Select station name as list element.
    dozen_stations = station_name * 12
    stations_flat = stations_flat + (dozen_stations)
    
    years = [precip_data.iloc[row,1]] # Select year.
    dozen_years = years * 12
    years_flat = years_flat + (dozen_years)


In [12]:
# Convert the lists to a dictionary.
data = {'Station': stations_flat,
        'Year': years_flat,
        'Month': months_flat,
        'Precipitation': precips_flat}

# Convert the dictionary to a completely flattened dataframe.
precip_flat = pd.DataFrame(data)
precip_flat.head(2)

,Station,Year,Month,Precipitation
0,AMH307,1997,1,NaN
1,AMH307,1997,2,NaN


Finally, I will index the flattened table by station, year, and month so these varaibles can be mixed and matched by thier values to select data. This is useful when you know what value you want to find, but not where it is.

In [13]:
# Set multiple indices in this dataframe.
precip_multi = precip_flat.set_index(['Station', 'Year', 'Month'])
precip_multi.head(2)

Precipitation
Station Year Month               
AMH307  1997 1                NaN
             2                NaN

In [14]:
# Test if multi-level indexing works.

# precip_multi.loc[('AMH307', 'Precipitation')] # Select all records from station AMH307.
# precip_multi.loc[('AMH307', 2000), 'Precipitation'] # Select all records from station AMH307 in 1997.
# precip_multi.loc[('AMH307', 1997, 3), 'Precipitation'] # Select January record from station AMH307 in 1997.
# test = len(precip_multi.loc[('AMH307',)])/12 # Count years of data for station AMH307.
# print(f'Station AMH307 has {int(test)} years of data.') 

## Feature Engineering (Variable Construction)

I want to create several additional variables from this data to quantify elements such as record completeness and to identify patterns within the data.

In [15]:
# Look up the total months recorded and the first and last year reporting data.
# Count how many months reported were no-data-collected months.
# These will be new columns added to the unique station data.

n_months_NaN = []
n_months_total = []
first_year_reporting = []
last_year_reporting = []

station_list = unique_stations['STATION']
n_stations = len(station_list)

for station in range(0, n_stations):
    current_station = station_list.iloc[station] # String
    
    n_months = len(precip_multi.loc[(current_station, ) ]) # Integer
    n_months_total.append(n_months)
    
    stations_earliest_year = precip_flat[precip_flat['Station'] == current_station].min()
    stations_earliest_year = stations_earliest_year.iloc[1]
    first_year_reporting.append(stations_earliest_year)
    
    stations_latest_year = precip_flat[precip_flat['Station'] == current_station].max()
    stations_latest_year = stations_latest_year.iloc[1]
    last_year_reporting.append(stations_latest_year)
    
    n_nans = 0
    for month in range(0, n_months):
        current_month = (month % 12) + 1
        
        current_months_value = precip_multi.loc[(current_station, ), 'Precipitation'].iloc[current_month]
        if np.isnan(current_months_value) == True:
            n_nans = n_nans + 1
    n_months_NaN.append(n_nans)

# Add lists as new columns.
unique_stations['First_year_collected'] = first_year_reporting
unique_stations['Last_year_collected'] = last_year_reporting
unique_stations['Months_sampled'] = n_months_total
unique_stations['Months_not_reported'] = n_months_NaN


In [16]:
# Calculate percent of months reporting non-NaN data and add as column. 
percent_coverage = [1 - (n_months_NaN / n_months_total) for n_months_NaN, n_months_total in zip(n_months_NaN, n_months_total)]
unique_stations['Percent_months_reported'] = percent_coverage


## Export data

This process has left us several data products to work with:
- precip_flat : a dataframe reporting precipitation by month, year, and station.
- precip_multi : a multi-level-indexed dataframe containing the same data as precip_flat.
- unique_stations : a dataframe of unique stations and record metadata
- unique_basins : a dataframe of unique drainage catchment basins

These fours tables are exported as csv files for use in other areas of this analysis. 

In [64]:
# Save with indices - that's the point.
precip_multi.to_csv(r'C:\Users\15414\Documents\GitHub\MA Precip\MAPrecipData\Products\precip_multi.csv')

# Indices not preserved.
precip_flat.to_csv(r'C:\Users\15414\Documents\GitHub\MA Precip\MAPrecipData\Products\precip_flat.csv', index=False)
unique_stations.to_csv(r'C:\Users\15414\Documents\GitHub\MA Precip\MAPrecipData\Products\unique_stations.csv', index=False)
unique_basins.to_csv(r'C:\Users\15414\Documents\GitHub\MA Precip\MAPrecipData\Products\unique_basins.csv', index=False)
lynn_data_30yr.to_csv(r'C:\Users\15414\Documents\GitHub\MA Precip\MAPrecipData\Products\lynn_data_30yr.csv', index=False)


Statistical analyses are conducted and figures construsted in PII. Context and discussion in P1. 